# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: ** It is a classification type of problem, because the answer is not continuous value but one of two classes.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [12]:
# TODO: Calculate number of students
n_students = student_data.shape[0]


# TODO: Calculate number of features
n_features = student_data.shape[1]
 
#passed_counts = student_data['passed'].value_counts()
#print student_data['passed'].value_counts()

# TODO: Calculate passing students
n_passed = 0
for v in student_data['passed']:
    if v == 'yes':
        n_passed += 1

# TODO: Calculate failing students
n_failed = 0
for v in student_data['passed']:
    if v == 'no':
        n_failed += 1
        
# TODO: Calculate graduation rate
grad_rate = (float(n_passed)/float(n_students))*100  ###### this is not working


# Print the results

print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed) 
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 31
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [13]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [23]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

#not part of original functions - should keep yes or no?
# def preprocess_target(y):
    
#     output = pd.DataFrame(index = y.index)

#     # Investigate each feature column for the data
#     for col, col_data in y.iteritems():
        
#         # If data type is non-numeric, replace all yes/no values with 1/0
       
#         col_data = col_data.replace(['yes', 'no'], [1, 0])

       
#         # Collect the revised columns
#         output = output.join(col_data)
    
#     return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

# y_all = preprocess_target(y_all) # need to preprocess targets
# print "preprocessed target also"

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [24]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split


# TODO: Set the number of training points
num_train = int(n_students*0.75)

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
#X_train = None
#X_test = None
#y_train = None
#y_test = None
X_train,X_test,y_train,y_test = train_test_split(
      X_all, y_all, test_size=num_test, random_state=0)


# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])


#test distribution of y_train, y_test
# print y_test[0:5]
# print y_train[0:5]
# n_passed = 0
# for v in y_train:
#     if v == 'yes':
#         n_passed += 1
# print float(n_passed) / X_train.shape[0]     
 
# n_passed = 0
# for v in y_test:
#     if v == 'yes':
#         n_passed += 1
# print float(n_passed) / X_test.shape[0]     

Training set has 296 samples.
Testing set has 99 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: ** 

**Random Forest** - It was the best classifier to predict air quality in a 2012 Kaggle competition (Kaggle, 2012).

Random Forests have also been used in Ecology. One application was to model the distribution of different tree species under future climate scenarios (Prasad et al., 2006). Another application was to model nest sites for cavity nesting birds 
(Cutler et al., 2007).

Strengths - Works well on wide range of problems and have few parameters to tune (Biau & Scornet, 2016). Accuracy and ability to deal with small sample sizes and
high-dimensional feature spaces (Biau & Scornet, 2016). Handles multiclass well (Debreuve).

Weakness - Little is know about the mathematical properties of the model (Biau & Scornet, 2016).

It is a good model for the student intervention project because it performs well without knowledge of the domain. I found almost 10% variation in the best F1 score between the first try without setting random_state and later trials with a random_state assigned, which makes me question if the original good result I encountered is mostly related to the randomizer having picked a speacilly good seed.

Lakkaraju (2015) found that Random Forest consistently outperformed all other models. They describe how attribute importance is calculated based on algorithm, in order to derive their framework to assess student’s risk of adverse outcomes. They use Random Forest ensembles, SVM and logistic regression, as well, as AdaBoost and Decision Trees.


**Logistic Regression** - has been used in credit scoring (Baesens, et al., 2003; Bahnsen, et al., 2014). One example was the credit scoring for small business (Bensic et al., 2005). 

Strengths - Interpretability of model parameters and ease of use (Dreiseitl & Ohno-Machado, 2002).
Can interpret model coefficients as indicators of feature importance (Howbert, 2012).
Resistant to overfitting (Howbert, 2012).

Weakness - Linear decision boundaries (Howbert, 2012).
It requires large number of data points per predictor in order to achieve stable result. 

It is a good model for the student intervention project because of the interpretability of model parameters.

**Suport Vector Machines** - have been used in predicting Diabetes (Yu, et al., 2010).

Strengths -  Non-linear with kernels (Weston). Theoretically motivated (Weston).
Good generalization with limited training samples.

Weakness - Fit time complexity is quadratic on the number of samples makes it hard to scale to datasets with more than 20000 samples.

It is a good algorithm for the student intervention project because it can provide a model that generalizes well given the limited number of examples to train.

**References:**

Baesens, B., Van Gestel, T., Viaene, S., Stepanova, M., Suykens, J., & Vanthienen, J. (2003). Benchmarking state-of-the-art classification algorithms for credit scoring. Journal of the operational research society, 54(6), 627-635.
https://www.jstor.org/stable/4101754

Bahnsen, A. C., Aouada, D., & Ottersten, B. (2014). Example-dependent cost-sensitive logistic regression for credit scoring. In Machine Learning and Applications (ICMLA), 2014 13th International Conference on (pp. 263-269). IEEE.
https://www.researchgate.net/publication/272441908_Example-Dependent_Cost-Sensitive_Logistic_Regression_for_Credit_Scoring 

Bensic, M., Sarlija, N., & Zekic‐Susac, M. (2005). Modelling small‐business credit scoring by using logistic regression, neural networks and decision trees. Intelligent Systems in Accounting, Finance and Management, 13(3), 133-150.
https://www.researchgate.net/profile/Mirta_Bensic/publication/215990290_Modeling_Small_Business_Credit_Scoring_by_Using_Logistic_Regression_Neural_Networks_and_Decision_Trees/links/0deec536ddf9233eaf000000.pdf

Biau, G., & Scornet, E. (2016). A random forest guided tour. Test, 25(2), 197-227.
https://arxiv.org/pdf/1511.05741.pdf 

Cutler, D. R., Edwards, T. C., Beard, K. H., Cutler, A., Hess, K. T., Gibson, J., & Lawler, J. J. (2007). Random forests for classification in ecology. Ecology, 88(11), 2783-2792.
http://digitalcommons.usu.edu/cgi/viewcontent.cgi?article=1805&context=wild_facpub

Debreuve, E. An introduction to random forests.
http://perso.math.univ-toulouse.fr/motimo/files/2013/07/random-forest.pdf 

Dreiseitl, S., & Ohno-Machado, L. (2002). Logistic regression and artificial neural network classification models: a methodology review. Journal of biomedical informatics, 35(5), 352-359. 
http://www.sciencedirect.com/science/article/pii/S1532046403000340 

Howbert, J. (2012). Machine Learning: Logistic Regression.
http://courses.washington.edu/css490/2012.Winter/lecture_slides/05b_logistic_regression.pdf

Kaggle (2012). http://blog.kaggle.com/2012/05/01/chucking-everything-into-a-random-forest-ben-hamner-on-winning-the-air-quality-prediction-hackathon/ 

Lakkaraju, H., Aguiar, E., Shan, C., Miller, D., Bhanpuri, N., Ghani, R., & Addison, K. L. (2015). A machine learning framework to identify students at risk of adverse academic outcomes. In Proceedings of the 21th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (pp. 1909-1918). ACM.
https://pdfs.semanticscholar.org/96f7/c50664a97a5426fad0bb362bfa5e556851c2.pdf 

Prasad, A. M., Iverson, L. R., & Liaw, A. (2006). Newer classification and regression tree techniques: bagging and random forests for ecological prediction. Ecosystems, 9(2), 181-199.
http://avesbiodiv.mncn.csic.es/estadistica/bt3.pdf 

Weston, J. Support Vector Machine and Statistical Learning Theory Tutorial.
http://www.cs.columbia.edu/~kathy/cs4701/documents/jason_svm_tutorial.pdf

Yu, W., Liu, T., Valdez, R., Gwinn, M., & Khoury, M. J. (2010). Application of support vector machine modeling for prediction of common diseases: the case of diabetes and pre-diabetes. BMC Medical Informatics and Decision Making, 10(1), 1.
https://bmcmedinformdecismak.biomedcentral.com/articles/10.1186/1472-6947-10-16    






### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [42]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for ************* test set: {:.4f}.".format(predict_labels(clf, X_test, y_test)) #remove asteriks

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [78]:
# TODO: Import the three supervised learning models from sklearn
# from sklearn import model_A
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier

# from sklearn import model_B
# from sklearn import tree
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.linear_model import SGDClassifier
# from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
                               
# from skearln import model_C

# from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC

# TODO: Initialize the three models
#clf_A = GaussianNB()
#clf_B = tree.DecisionTreeClassifier()
#clf_A = AdaBoostClassifier()
#clf_A = BaggingClassifier(tree.DecisionTreeClassifier())
clf_A = RandomForestClassifier(random_state=11)

#clf_B = KNeighborsClassifier()
# clf_B = SGDClassifier()
# clf_B = SGDClassifier(loss="hinge", penalty="l2")
#clf_B = GradientBoostingClassifier(n_estimators=10)
#clf_B = GradientBoostingClassifier()
clf_B = LogisticRegression(random_state=42)

#clf_C =  RandomForestClassifier()
#clf_C =  RandomForestClassifier(n_estimators=100)
clf_C = SVC(random_state=42)

# TODO: Set up the training set sizes
X_train_100 = X_train[0:100]
y_train_100 = y_train[0:100]
#print X_train_100.shape

X_train_200 = X_train[0:200]
y_train_200 = y_train[0:200]
#print X_train_200.shape

X_train_300 = X_train[0:296]
y_train_300 = y_train[0:296]
#print X_train_300.shape

# TODO: Execute the 'train_predict' function for each classifier and each training set size
#train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100,X_test,y_test)
train_predict(clf_A, X_train_200, y_train_200,X_test,y_test)
train_predict(clf_A, X_train_300, y_train_300,X_test,y_test)

train_predict(clf_B, X_train_100, y_train_100,X_test,y_test)
train_predict(clf_B, X_train_200, y_train_200,X_test,y_test)
train_predict(clf_B, X_train_300, y_train_300,X_test,y_test)


train_predict(clf_C, X_train_100, y_train_100,X_test,y_test)
train_predict(clf_C, X_train_200, y_train_200,X_test,y_test)
train_predict(clf_C, X_train_300, y_train_300,X_test,y_test)

Training a RandomForestClassifier using a training set size of 100. . .
Trained model in 0.0568 seconds
Made predictions in 0.0079 seconds.
F1 score for training set: 0.9841.
Made predictions in 0.0063 seconds.
F1 score for ************* test set: 0.7442.
Training a RandomForestClassifier using a training set size of 200. . .
Trained model in 0.0426 seconds
Made predictions in 0.0057 seconds.
F1 score for training set: 0.9810.
Made predictions in 0.0058 seconds.
F1 score for ************* test set: 0.7077.
Training a RandomForestClassifier using a training set size of 296. . .
Trained model in 0.0342 seconds
Made predictions in 0.0059 seconds.
F1 score for training set: 0.9877.
Made predictions in 0.0057 seconds.
F1 score for ************* test set: 0.7778.
Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0061 seconds
Made predictions in 0.0006 seconds.
F1 score for training set: 0.8000.
Made predictions in 0.0002 seconds.
F1 score for ************

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0568                 |             0.0063           |      0.9841            |         0.7442        |
| 200               |        0.0426          |              0.0058          |        0.9810          |       0.7077          |
| 296               |           0.0342              |            0.0057            |        0.9877          |    0.7778      |

** Classifer 2 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0061                |         0.0002               |      0.8000            |    0.7153              |
| 200               |     0.0029             |          0.0002              |     0.8269             |   0.7770               |
| 296               |   0.0028                      |     0.0002                   |   0.8474               |     0.7801     |

** Classifer 3 - Support Vector Machine**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |      0.0013                   |     0.0008                   |   0.8750               |    0.7662             |
| 200               |     0.0040                    |    0.0022                    |  0.8771                |  0.7632|
| 296               |    0.0094                     |  0.0020                      |  0.8712                |  0.7632               |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Logistic Regression provided the best time performance for training and prediction for the student intervention dataset. It had the best F1 scores, which consistently improved as the training sample increased. Both Random Forest and SVM had more variability in the F1 scores and did not increase consistently with the size of the training sample.

On a previous run without a random seed, the Random Forest had a better result (0.7883, 0.6912, 0,8085) but subsequent runs did not match this promissing result.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

The Logistic Regression model uses data from students' previous records to calculate the importance of each feature of a student to predict student's success. The relative importance of these features  can then be used against new student data to predict the outcome of a student. The prediction could be seen as the odds a student is likely to pass the final exam.


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [94]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

# TODO: Create the parameters list you wish to tune
#parameters = {'C': [0.25,0.5,0.75, 1.0]}

parameters = { 'C': [0.25,0.5,0.75, 1.0],'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag']}
#Best parameters were {'C': 0.75, 'solver': 'sag'}. testing F1 score of 0.8000.


# TODO: Initialize the classifier

clf = LogisticRegression(random_state=42)

# TODO: Make an f1 scoring function using 'make_scorer' 

f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method

grid_obj = GridSearchCV(clf, parameters, f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)
print "Best parameters were {}.".format(grid_obj.best_params_ )
print
print "Logistic expression coefficients"
 

# Get the estimator
clf = grid_obj.best_estimator_
 


# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

 

Best parameters were {'C': 0.75, 'solver': 'sag'}.

Logistic expression coefficients
Made predictions in 0.0005 seconds.
Tuned model has a training F1 score of 0.8267.
Made predictions in 0.0003 seconds.
Tuned model has a testing F1 score of 0.8000.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final model's F1 score is **0.8000** for testing and 0.8267 for training. As compared to **0.7801** and 0.8474, respectivelly. This shows that some regularization (C=0.75) helped to generalize the model. The default solver Liblinear appears to be less appropriate than **sag** for this data set and test/training split during the Grid Search Cross Validation.

Below are the final coefficients:

| Feature | Coeff | Feature | Coeff |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: | 
|  school_GP 	|	 0.0651  |  school_MS	| 	0.0351  | 
| sex_F	|	-0.0763 |  sex_M 	|	0.1764  |
| age    |    -0.0042 | addressR    |    -0.1066 |
| address |      0.2068  | famsize_GT3    |    -0.0140 |
| famsize_LE3    |    0.1142 | Pstatus_A    |    0.1410 |
 |  Pstatus_T    |    -0.0409  | Med    |    0.0850 |
|  Fed    |    0.1405 | Mjob_at_home    |    -0.0417  |  
|  **Mjob_health**     |  0.2663  |  Mjob_other    |    -0.1161  |  
| **Mjob_services**    |    0.2812  |  **Mjob_teacher**    |    -0.2895  | 
| Fjob_at_home    |    0.0490  |  Fjob_health    |    0.0350  |  
|  Fjob_other    |    -0.0223  |  Fjob_services    |    -0.0019  |  
|  Fjob_teacher    |    0.0404  |  reason_course    |    -0.1517  |  
|  reason_home    |    0.0855  |   reason_other    |    0.0622  |  
| reason_reputation    |    0.1042  |  guardian_father    |    0.1233  |  
|  guardian_mother    |    0.1216  |  guardian_other    |    -0.1448  | 
|traveltime    |    -0.0127  |  **studytime**    |    **0.4015**  | 
  |  **failures**    |    **-0.7234** |  **schoolsup**    |    -0.2143    |
   |   **famsup**    |    -0.3145 |  paid    |    0.0703   |  
 |    activities    |  0.0206  |  nursery    |    -0.1450  |   
|   **higher**    |    0.2675  |  internet    |    0.2546  |  
 |   romantic    |    -0.1957 |   famrel    |    0.0939  |  
 |  freetime    |    0.0859 |  **goout**    |    -0.4705  |   
|  Dalc    |    -0.0895 |  **Walc**    |    0.3369  |  
|  health    |    -0.1017 |  absences    |    -0.0337  |  

 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.